In [15]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from matplotlib import verbose
#verbose.level = 'helpful'      # one of silent, helpful, debug, debug-annoying

# taken from https://github.com/jrmontag/STLDecompose
# requires installation - pip install stldecompose
#from stldecompose import decompose
# Simpler smoothing uses:
# from statsmodels.tsa.seasonal import seasonal_decompose
#import libraries required for analysis
#set project id for BigQuery
projectid = "ebmdatalab"
import geopandas as gpd

In [46]:
ghost_df = pd.read_gbq("""
SELECT
  prac.ccg_id,
  rx.practice_code,
  rx.bnf_code,
  rx.bnf_description,
  rx.items,
  rx.quantity,
  rx.nic,
  dt.concession,
  dt.quantity as dt_quantity,
  dt.price_pence,

  SUM(rx.items*rx.quantity) AS QI,
  ROUND(IEEE_DIVIDE(rx.NIC,(rx.items*rx.quantity)),4) AS rx_ppu,
  ROUND(CASE
      WHEN dt.concession IS NOT NULL THEN IEEE_DIVIDE((dt.concession/100),dt.quantity)
      ELSE IEEE_DIVIDE((dt.price_pence/100),dt.quantity) END,4) AS dt_ppu,
  rx.quantity*rx.items*(ROUND(IEEE_DIVIDE(rx.NIC,(rx.items*rx.quantity)),4)- ROUND(CASE
        WHEN dt.concession IS NOT NULL THEN IEEE_DIVIDE((dt.concession/100),dt.quantity)
        ELSE IEEE_DIVIDE((dt.price_pence/100),dt.quantity) END,4)) AS excess_cost
FROM
  richard.prescribing_2018_09_full AS rx
JOIN
  dmd.dt_viewer AS dt
ON
  rx.bnf_code=dt.bnf_code
  join 
  hscic.practices as prac
  on
  rx.practice_code=prac.code  
WHERE
  (rx.bnf_description LIKE '%_Tab%'
    OR rx.bnf_description LIKE '%_Cap%')
  AND dt.date='2018-09-01' AND rx.bnf_code not like '0410020C0%AC'
GROUP BY
  prac.ccg_id,
  rx.practice_code,
  rx.bnf_code,
  rx.bnf_description,
  rx.quantity,
  rx.items,
  rx_ppu,
  dt_ppu,
  dt.concession,
  dt.quantity,
  dt.price_pence,
  rx.NIC
HAVING
  rx_ppu <> dt_ppu
  order by excess_cost desc
""", projectid, dialect='standard')
 
ghost_df.to_csv("Ghost Branded Generics.csv")

Requesting query... ok.
Query running...
  Elapsed 11.07 s. Waiting...
  Elapsed 21.56 s. Waiting...
Query done.
Processed: 1.3 Gb

Retrieving results...
  Got page: 1; 1% done. Elapsed 26.31 s.
  Got page: 2; 2% done. Elapsed 29.2 s.
  Got page: 3; 3% done. Elapsed 32.25 s.
  Got page: 4; 4% done. Elapsed 35.02 s.
  Got page: 5; 5% done. Elapsed 37.71 s.
  Got page: 6; 6% done. Elapsed 40.35 s.
  Got page: 7; 7% done. Elapsed 43.33 s.
  Got page: 8; 8% done. Elapsed 46.14 s.
  Got page: 9; 9% done. Elapsed 48.74 s.
  Got page: 10; 10% done. Elapsed 51.5 s.
  Got page: 11; 11% done. Elapsed 54.25 s.
  Got page: 12; 12% done. Elapsed 57.23 s.
  Got page: 13; 13% done. Elapsed 60.18 s.
  Got page: 14; 14% done. Elapsed 63.21 s.
  Got page: 15; 15% done. Elapsed 66.75 s.
  Got page: 16; 16% done. Elapsed 69.66 s.
  Got page: 17; 17% done. Elapsed 72.35 s.
  Got page: 18; 18% done. Elapsed 75.34 s.
  Got page: 19; 19% done. Elapsed 79.03 s.
  Got page: 20; 20% done. Elapsed 83.04 s.
  Got 

In [49]:
gpd_ghost_df = ghost_df.groupby('ccg_id')['excess_cost'].sum().reset_index().sort_values('excess_cost',ascending=False)

In [50]:
gpd_ghost_df.head(20)

,ccg_id,excess_cost
95,06H,36167.2935
198,15F,26274.7840
197,15E,25910.4207
177,11J,24347.8120
83,05L,22618.1792
224,99P,22106.7665
178,11M,21685.3784
94,06F,20982.5777
96,06K,19156.1250
56,03T,18179.7449


In [43]:
gpd_ghost_df.sum()

bnf_description    Paracet_Tab 500mgAtorvastatin_Tab 20mgMemantin...
excess_cost                                                   966716
dtype: object

In [ ]:
ghost_df.head()